# Named Entity Recognition

# 1. Russian Language

## Yargy
Для начала посмотрим на то, что скрывает под капотом Yargy-парсер.

Yargy-парсер — **парсер, основанный на правилах, в котором правила для извлечения сущностей описываются с помощью контекстно-свободных грамматик и словарей.**

Подробнее про Yargy-парсер можно почитать в [этом](https://habr.com/ru/post/349864/) посте.

GitHub [Yargy](https://github.com/natasha/yargy)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install yargy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 53.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=bccbc437b759eafd5969478937fee2ec641f3a5981e74073588cbc5046ea7f58
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from yargy import Parser, rule, and_, not_
from yargy.interpretation import fact
from yargy.predicates import gram
from yargy.relations import gnc_relation
from yargy.pipelines import morph_pipeline

В Yargy-парсер уже заложена большая часть правил по выявлению именованных сущностей. Для примера рассмотрим, как выглядит правило для имен людей.

In [ ]:
# хорошо для специфических данных

# Определяем факт 'Name' с полями 'first' (имя) и 'last' (фамилия)
Name = fact(
    'Name',
    ['first', 'last'],
)

# Определяем факт 'Person' с полями 'position' (должность) и 'name' (структура 'Name')
Person = fact(
    'Person',
    ['position', 'name']
)

# Правило для фамилии: должна быть сущ. (Surn), но не аббревиатурой
LAST = and_(
    gram('Surn'),  # фамилия (существительное с тегом "Surn")
    not_(gram('Abbr')),  # не аббревиатура
)

# Правило для имени: должно быть имя (Name), но не аббревиатура
FIRST = and_(
    gram('Name'),  # имя (существительное с тегом "Name")
    not_(gram('Abbr')),  # не аббревиатура
)

# Создаем морфологический пайплайн для списка должностей
POSITION = morph_pipeline([
    'управляющий директор',  # должность 1
    'вице-мэр'  # должность 2
])

In [ ]:
# Создаем отношение рода/числа/падежа (gnc_relation) для согласования между частями речи
gnc = gnc_relation()

In [ ]:
# Правило для полного имени: сначала имя, затем фамилия, оба согласованы по роду/числу/падежу
NAME = rule(
    FIRST.interpretation(# Интерпретируем имя как поле 'first' в структуре 'Name'
        Name.first
    ).match(gnc),  # Должно быть согласовано по род/число/падеж
    LAST.interpretation(  # Интерпретируем фамилию как поле 'last' в структуре 'Name'
        Name.last
    ).match(gnc)  # Согласование с именем
).interpretation(
    Name  # Весь результат интерпретируется как структура 'Name'
)

# Правило для персоны: должность + имя, согласованные по роду/числу/падежу
PERSON = rule(
    POSITION.interpretation(  # Интерпретируем должность как поле 'position' в структуре 'Person'
        Person.position
    ).match(gnc),  # Согласование с именем
    NAME.interpretation(  # Интерпретируем имя как поле 'name' в структуре 'Person'
        Person.name
    )
).interpretation(  # Весь результат интерпретируется как структура 'Person'
    Person
)

In [ ]:
# Создаем парсер, который будет искать в тексте по правилу PERSON (должность + имя)
parser = Parser(PERSON)

In [ ]:
# Ищем точное совпадение в тексте с должностью и именем
match = parser.match('управляющий директор Иван Ульянов')
print(match)

Match(tokens=[MorphToken(value='управляющий', span=[0, 11), type='RU', forms=[Form('управлять', Grams(PRTF,Subx,actv,impf,intr,masc,nomn,pres,sing))]), MorphToken(value='директор', span=[12, 20), type='RU', forms=[Form('директор', Grams(NOUN,anim,masc,nomn,sing))]), MorphToken(value='Иван', span=[21, 25), type='RU', forms=[Form('иван', Grams(NOUN,Name,anim,masc,nomn,sing))]), MorphToken(value='Ульянов', span=[26, 33), type='RU', forms=[Form('ульянов', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])], span=[0, 33))


In [ ]:
text = 'За свою трудовую деятельность управляющий директор Иван Ульянов сделал огромный вклад в развитие завода'

In [ ]:
# Ищем все вхождения, которые соответствуют правилу PERSON, и выводим значения токенов (слов)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['управляющий', 'директор', 'Иван', 'Ульянов']


**Немного украшательства**

Библиотека ipymarkup предназначена для красивой визуализации NER.

In [ ]:
!pip install ipymarkup

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26096 sha256=c1d15d095e34a6460eb5e6f0dfb5e3466e13d2746f757feac6eea911395ff83d
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [ ]:
from ipymarkup import show_span_box_markup as show_markup # We will use this alisas for different markups!

In [ ]:
matches = parser.findall(text)
spans = [_.span for _ in matches]
spans

[[30, 63)]

In [ ]:
text[30:63]

'управляющий директор Иван Ульянов'

In [ ]:
show_markup(text, spans)



```
# Als Code formatiert
```

## Slovnet by Natasha (real world library)

[Slovnet](https://habr.com/ru/post/516098/) — проект по обучению нейросетевых моделей для обработки естественного русского языка. В библиотеке собраны качественные **компактные модели** для извлечения именованных сущностей, разбора морфологии и синтаксиса.

[Гитхаб](https://github.com/natasha/slovnet) проекта.

SlovNet is a Python library for deep-learning based NLP modeling for Russian language. Library is integrated with other Natasha projects: Nerus — large automatically annotated corpus, Razdel — sentence segmenter, tokenizer and Navec — compact Russian embeddings. Slovnet provides high quality practical models for Russian NER, morphology and syntax, see evaluation section for more:

* **NER in Slovnet is 1-2% worse than current BERT SOTA by DeepPavlov but 60 times smaller in size (\~30 MB) and works fast on CPU (\~25 news articles/sec).**

* Morphology tagger and syntax parser have comparable accuracy on news dataset with large SOTA BERT models, take 50 times less space (\~30 MB), work faster on CPU (\~500 sentences/sec).

In [ ]:
!pip install navec
!pip install slovnet

In [ ]:
from navec import Navec  # библиотека для работы с векторными представлениями слов (word embeddings)
from slovnet import NER  # библиотека для распознавания сущностей (NER)
from ipymarkup import show_span_ascii_markup as show_markup

In [ ]:
text = '''Впервые в мире нейросеть от ПАО Сбербанк написала сборник рассказов вместе с писателем Павлом Пепперштейном, который выпустило издательство Individuum
В издательстве Individuum вышел сборник рассказов «Пытаясь проснуться», написанных писателем и художником Павлом Пепперштейном и генеративной нейросетью ruGPT-3, разработанной командой SberDevices. Бумажную книгу уже сейчас можно заказать в интернет-магазине издательства, а на полках книжных магазинов она появится до конца мая. Электронная версия с сегодняшнего дня доступна эксклюзивно на «Букмейте».'''

In [ ]:
text

'Впервые в мире нейросеть от ПАО Сбербанк написала сборник рассказов вместе с писателем Павлом Пепперштейном, который выпустило издательство Individuum\nВ издательстве Individuum вышел сборник рассказов «Пытаясь проснуться», написанных писателем и художником Павлом Пепперштейном и генеративной нейросетью ruGPT-3, разработанной командой SberDevices. Бумажную книгу уже сейчас можно заказать в интернет-магазине издательства, а на полках книжных магазинов она появится до конца мая. Электронная версия с сегодняшнего дня доступна эксклюзивно на «Букмейте».'

Для того, чтобы выявить в тексте именованные сущности воспользуемся **предобученными эмбеддингами** из библиотеки [Navec](https://github.com/natasha/navec), которые мы **затем будем использовать в качестве input'а для предобученной модели из Slovnet.**

Библиотека [Navec](https://natasha.github.io/navec/) — часть проекта [Natasha](https://github.com/natasha), коллекция предобученных эмбеддингов для русского языка.

Скачаем предобученную [модель](https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar) `'navec_news_v1_1B_250K_300d_100q.tar'`.

In [ ]:
# needed for google drive file imports in google colab
# skip, if executed locally
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
# use if on google colab
# navec = Navec.load(r'/content/drive/My Drive/OTUS/NER/navec_news_v1_1B_250K_300d_100q.tar')
# use locally with your path

# предобученные эмбеддинги
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

А теперь, **используя предобученные эмбеддинги из Navec найдем в тексте именованые сущности с помощью предобученной модели  slovnet.**

In [ ]:
# use if on google colab
# ner = NER.load(r'/content/drive/My Drive/OTUS/NER/slovnet_ner_news_v1.tar')
# use locally with your path

# Russian NER model, standart PER, LOC, ORG annotation, trained on news articles.
ner = NER.load('slovnet_ner_news_v1.tar')

In [ ]:
# init
ner.navec(navec);

In [ ]:
# apply NER
markup = ner(text)

In [ ]:
markup

SpanMarkup(
    text='Впервые в мире нейросеть от ПАО Сбербанк написала сборник рассказов вместе с писателем Павлом Пепперштейном, который выпустило издательство Individuum\nВ издательстве Individuum вышел сборник рассказов «Пытаясь проснуться», написанных писателем и художником Павлом Пепперштейном и генеративной нейросетью ruGPT-3, разработанной командой SberDevices. Бумажную книгу уже сейчас можно заказать в интернет-магазине издательства, а на полках книжных магазинов она появится до конца мая. Электронная версия с сегодняшнего дня доступна эксклюзивно на «Букмейте».',
    spans=[Span(
         start=28,
         stop=40,
         type='ORG'
     ),
     Span(
         start=87,
         stop=107,
         type='PER'
     ),
     Span(
         start=140,
         stop=150,
         type='ORG'
     ),
     Span(
         start=166,
         stop=176,
         type='ORG'
     ),
     Span(
         start=257,
         stop=277,
         type='PER'
     ),
     Span(
         start=3

In [ ]:
show_markup(markup.text, markup.spans)

Впервые в мире нейросеть от ПАО Сбербанк написала сборник рассказов 
                            ORG─────────                            
вместе с писателем Павлом Пепперштейном, который выпустило 
                   PER─────────────────                    
издательство Individuum
             ORG───────
В издательстве Individuum вышел сборник рассказов «Пытаясь 
               ORG───────                                  
проснуться», написанных писателем и художником Павлом Пепперштейном и 
                                               PER─────────────────   
генеративной нейросетью ruGPT-3, разработанной командой SberDevices. 
                                                        ORG────────  
Бумажную книгу уже сейчас можно заказать в интернет-магазине 
издательства, а на полках книжных магазинов она появится до конца мая.
 Электронная версия с сегодняшнего дня доступна эксклюзивно на 
«Букмейте».
 ORG─────  


### Визуализация сущностей

In [ ]:
from ipymarkup import show_span_box_markup as show_markup

In [ ]:
show_markup(text, markup.spans)

«Пытаясь проснуться» и ruGPT-3, не классифицированы.

-> **можно использовать микс инструментов и взять объединение всех результатов**

# 2. English Language

А что с английским языком? Модели от navec и slovnet обучены для русского языка.

Для английского воспользуемся библиотекой [spacy](https://spacy.io/), в которую уже встроен парсер для английского языка: [en_core_web_sm](https://spacy.io/models/en).

In [ ]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 85.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy import displacy # NER rendering
from collections import Counter
import en_core_web_sm # NER model

In [ ]:
# download model
# when choosing a model think of the accuracy vs performance tradeoff (small vs large model)
nlp = en_core_web_sm.load()

Посмотрим, как работает английский NER из spacy на примере с новостного сайта.

In [ ]:
text = '''One day in mid-November, workers at OpenAI got an unexpected assignment: Release a chatbot, fast. The chatbot, an executive announced, would be known as “Chat with GPT 3.5,” and it would be made available free to the public. '''

In [ ]:
doc = nlp(text)
doc

One day in mid-November, workers at OpenAI got an unexpected assignment: Release a chatbot, fast. The chatbot, an executive announced, would be known as “Chat with GPT 3.5,” and it would be made available free to the public. 

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
# show entities
doc.ents

(One day, mid-November, OpenAI, GPT)

In [ ]:
[(X.text, X.label_) for X in doc.ents]

[('One day', 'DATE'),
 ('mid-November', 'DATE'),
 ('OpenAI', 'GPE'),
 ('GPT', 'ORG')]

In [ ]:
# IOB annotation format
[(X, X.ent_iob_, X.ent_type_) for X in doc]

[(One, 'B', 'DATE'),
 (day, 'I', 'DATE'),
 (in, 'O', ''),
 (mid, 'B', 'DATE'),
 (-, 'I', 'DATE'),
 (November, 'I', 'DATE'),
 (,, 'O', ''),
 (workers, 'O', ''),
 (at, 'O', ''),
 (OpenAI, 'B', 'GPE'),
 (got, 'O', ''),
 (an, 'O', ''),
 (unexpected, 'O', ''),
 (assignment, 'O', ''),
 (:, 'O', ''),
 (Release, 'O', ''),
 (a, 'O', ''),
 (chatbot, 'O', ''),
 (,, 'O', ''),
 (fast, 'O', ''),
 (., 'O', ''),
 (The, 'O', ''),
 (chatbot, 'O', ''),
 (,, 'O', ''),
 (an, 'O', ''),
 (executive, 'O', ''),
 (announced, 'O', ''),
 (,, 'O', ''),
 (would, 'O', ''),
 (be, 'O', ''),
 (known, 'O', ''),
 (as, 'O', ''),
 (“, 'O', ''),
 (Chat, 'O', ''),
 (with, 'O', ''),
 (GPT, 'B', 'ORG'),
 (3.5, 'O', ''),
 (,, 'O', ''),
 (”, 'O', ''),
 (and, 'O', ''),
 (it, 'O', ''),
 (would, 'O', ''),
 (be, 'O', ''),
 (made, 'O', ''),
 (available, 'O', ''),
 (free, 'O', ''),
 (to, 'O', ''),
 (the, 'O', ''),
 (public, 'O', ''),
 (., 'O', '')]

In [ ]:
displacy.render(doc, jupyter=True, style='ent')

А теперь давайте возьмем реальный текст [новости](https://www.nytimes.com/2022/04/21/sports/basketball/nets-celtics-kevin-durant.html) с новостного сайта [NY Times](https://www.nytimes.com) и посмотрим, какие именованные сущности выявит spacy.


Посмотрим, **о ком и о чем эта статья**.

In [ ]:
ny_bb = 'BOSTON — Kevin Durant had no room. He admitted as much. Whenever he had the ball against the Celtics on Wednesday night, and even when he did not, defenders were crowding his space, shadowing him, draping themselves all over him like Saran wrap. They were on the perimeter, and in the paint, and at the elbow. How was it possible that only five of them were on the court at once? “They’re mucking up actions when I run off stuff,” said Durant, who singled out the Celtics’ Al Horford for “leaving his man to come over and hit me sometimes.” Durant went on: “Just two or three guys hitting me wherever I go. And that’s just the nature of the beast in the playoffs.” It was nearing 11 p.m. as Durant offered up his post-mortem of the Nets’ 114-107 loss to the Celtics in Game 2 of their first-round playoff series, and he did not necessarily seem concerned. In fact, his analysis came off as dispassionate: Here were the facts, and it was his job to remedy the issues as the Nets seek to rebound from their two-games-to-none deficit in the best-of-seven series. It heads to Brooklyn for Game 3 on Saturday night. “It’s on me to just finish it and figure it out,” he said. “I’m not expecting my teammates or the defense to give me anything. I just got to go out there and play.”'

In [ ]:
print(ny_bb)

BOSTON — Kevin Durant had no room. He admitted as much. Whenever he had the ball against the Celtics on Wednesday night, and even when he did not, defenders were crowding his space, shadowing him, draping themselves all over him like Saran wrap. They were on the perimeter, and in the paint, and at the elbow. How was it possible that only five of them were on the court at once? “They’re mucking up actions when I run off stuff,” said Durant, who singled out the Celtics’ Al Horford for “leaving his man to come over and hit me sometimes.” Durant went on: “Just two or three guys hitting me wherever I go. And that’s just the nature of the beast in the playoffs.” It was nearing 11 p.m. as Durant offered up his post-mortem of the Nets’ 114-107 loss to the Celtics in Game 2 of their first-round playoff series, and he did not necessarily seem concerned. In fact, his analysis came off as dispassionate: Here were the facts, and it was his job to remedy the issues as the Nets seek to rebound from t

In [ ]:
article = nlp(ny_bb)
len(article.ents)

17

In [ ]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'GPE': 2,
         'PERSON': 3,
         'DATE': 2,
         'PRODUCT': 1,
         'CARDINAL': 5,
         'TIME': 2,
         'ORG': 1,
         'ORDINAL': 1})

In [ ]:
items = [x.text for x in article.ents]
Counter(items).most_common(10)

[('Durant', 2),
 ('two', 2),
 ('BOSTON', 1),
 ('Kevin Durant', 1),
 ('Wednesday', 1),
 ('Saran', 1),
 ('only five', 1),
 ('three', 1),
 ('11 p.m.', 1),
 ('114', 1)]

In [ ]:
sentences = [x for x in article.sents]
print(sentences[3])

They were on the perimeter, and in the paint, and at the elbow.


И конечно красивая разметка.

In [ ]:
displacy.render(nlp(str(sentences)), jupyter=True, style='ent')

# FAQ

### Почему не обучаем свою модель в практике?

Есть смысл только начиная с BERT